In [ ]:
from transformers import AutoTokenizer, pipeline, AutoProcessor, WhisperForConditionalGeneration, WhisperFeatureExtractor, WhisperProcessor
from peft import PeftModel
import torch
import os
import transformers
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device=torch.device("cuda")
torch_dtype="auto"#torch.float32,
transformers.utils.logging.set_verbosity_error()
generate_kwargs = {"language":"<|ja|>","task":"translate"}#,"temperature":"0.0"}
sample = "01.wav"

In [ ]:
base = "openai/whisper-small"
base_model = WhisperForConditionalGeneration.from_pretrained(base, torch_dtype="auto", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(base)
processor = AutoProcessor.from_pretrained(base)
feature_extractor = WhisperFeatureExtractor.from_pretrained(base)

pipe = pipeline("automatic-speech-recognition", base_model, tokenizer=tokenizer, feature_extractor=feature_extractor, torch_dtype="auto")
result=pipe(sample, generate_kwargs=generate_kwargs)
print(result)

In [ ]:
peft_model = "./sin2piusc/whisper-small_tran_1-test/checkpoint-1000"
model_1 = PeftModel.from_pretrained(base_model, peft_model, is_trainable=True, torch_dtype="auto")
pipe_2 = pipeline("automatic-speech-recognition", model_1, tokenizer=tokenizer, feature_extractor=feature_extractor, torch_dtype="auto")
result_2=pipe_2(sample, generate_kwargs=generate_kwargs)
print(result_2)

In [ ]:
weighted_adapter_name = "4"
model_1.load_adapter("sin2piusc/whisper-med_15k", adapter_name="a0")
model_1.load_adapter("sin2piusc/whisper-med_22k", adapter_name="a1")
model_1.load_adapter("./adapters/whisper-medium-5k", adapter_name="a2")#./sin2piusc/whisper-medium_s1/checkpoint-1200
model_1.load_adapter("sin2piusc/whisper-medium-anime-5k", adapter_name="a3")
model_1.add_weighted_adapter(
    adapters=["a0", "a1", "a2", "a3"],
    weights=[0.0, 0.0, 0.25, 0.25],
    adapter_name=weighted_adapter_name,
    combination_type="cat",
    )

model_1.set_adapter("4")

In [ ]:
pipe = pipeline("automatic-speech-recognition", model_1, tokenizer=tokenizer, feature_extractor=feature_extractor, torch_dtype="auto")
result=pipe(sample, generate_kwargs=generate_kwargs)#, chunk_length_s=10, stride_length_s=[2,2])
print(result)

In [ ]:
out = "./saved_models/whisper-small_tran_1-test"
tokenizer.save_pretrained(out +"/Tokenizer")
processor.save_pretrained(out + "/Processor")
model_1.save_pretrained(out + "/Adapter")
model_1.merge_and_unload().save_pretrained(out + "/Merged_Model")

# unmerge the LoRA layers from the base model
# model.unmerge_adapter()